## 데이터 집계
* 갭마인더 메소드로 평균값 구하기

In [1]:
import pandas as pd
df = pd.read_csv('./gapminder.tsv', sep = '\t')

In [9]:
# year 열 기준으로 그룹화한 다음 lifeExp 열의 평균 구하기
avg_life_exp_by_year = df.groupby('year')['lifeExp'].mean()
avg_life_exp_by_year

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

* groupby 메서드이 분할/반영/결합 과정 확인

In [4]:
# 분할 작업
years = df['year'].unique()    # df.year.unique() 동일
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [5]:
# 반영 작업
year_means = []
for y in years :
    m = df.loc[df.year == y, :].lifeExp.mean()
    year_means.append((y,m))
    
print(year_means)

[(1952, 49.05761971830987), (1957, 51.507401126760534), (1962, 53.60924901408449), (1967, 55.67828957746479), (1972, 57.647386478873244), (1977, 59.57015746478873), (1982, 61.53319718309858), (1987, 63.21261267605636), (1992, 64.16033802816901), (1997, 65.01467605633805), (2002, 65.69492253521126), (2007, 67.00742253521126)]


In [6]:
# 결합 작업
df2 = pd.DataFrame({
    'year' : [y for y,m in year_means],
    '' : [m for y,m in year_means]
})

df2

,year,
0,1952,49.057620
1,1957,51.507401
2,1962,53.609249
3,1967,55.678290
4,1972,57.647386
5,1977,59.570157
6,1982,61.533197
7,1987,63.212613
8,1992,64.160338
9,1997,65.014676


* 사용자 함수 적용 agg() : 입력받은 열의 평균값을 구하는 사용자 함수(apply()와 유사)

In [10]:
def my_mean(values) :
    n = len(values)
    sum = 0
    for value in values :
        sum += value
    return sum / n

In [15]:
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
agg_my_mean

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

* 2개 인자 사용

In [16]:
# 연도별 평균 수명에서 전체 평균 수명 차이를 구하는 사용자 함수
def my_mean_diff(values, diff_value) :
    n = len(values)
    sum = 0
    for value in values :
        sum += value
    mean = sum / n
    return mean - diff_value

In [17]:
# 전체 평균 수명
global_mean = df.lifeExp.mean()
global_mean

59.47443936619714

In [19]:
# agg() 적용
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value = global_mean)
agg_mean_diff

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64

### 여러 개의 집계 메소드 한번에 적용(리스트)

In [22]:
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
gdf

,count_nonzero,mean,std
year,,,
1952,142,49.057620,12.225956
1957,142,51.507401,12.231286
1962,142,53.609249,12.097245
1967,142,55.678290,11.718858
1972,142,57.647386,11.381953
1977,142,59.570157,11.227229
1982,142,61.533197,10.770618
1987,142,63.212613,10.556285
1992,142,64.160338,11.227380


In [23]:
# 여러 개의 집계 메소드를 여러 개의 열에 적용
gdf_dict = df.groupby('year').agg(
{'lifeExp' : 'mean', 'pop' :'median', 'gdpPercap':'median'})
gdf_dict

,lifeExp,pop,gdpPercap
year,,,
1952,49.057620,3943953.0,1968.528344
1957,51.507401,4282942.0,2173.220291
1962,53.609249,4686039.5,2335.439533
1967,55.678290,5170175.5,2678.334740
1972,57.647386,5877996.5,3339.129407
1977,59.570157,6404036.5,3798.609244
1982,61.533197,7007320.0,4216.228428
1987,63.212613,7774861.5,4280.300366
1992,64.160338,8688686.5,4386.085502


## 데이터 변환

* agg() : year 기준으로 묶어서 통계값을 반환
* transform() : 통계컬럼을 이용하여 다시 각각의 요소들로 반환

In [24]:
# 표준점수 계산 사용자 함수
def my_zscore(x) :
    return (x - x.mean()) / x.std()

In [26]:
# 각 연도별 lifeExp 열의 표준점수 계산
trans_z = df.groupby('year').lifeExp.transform(my_zscore)
print(trans_z)

0      -1.656854
1      -1.731249
2      -1.786543
3      -1.848157
4      -1.894173
          ...   
1699   -0.081621
1700   -0.336974
1701   -1.574962
1702   -2.093346
1703   -1.948180
Name: lifeExp, Length: 1704, dtype: float64


### 누락값을 평균값으로 처리하기

In [27]:
import seaborn as sns
import numpy as np

np.random.seed(1234)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10

,total_bill,tip,sex,smoker,day,time,size
187,30.46,2.00,Male,Yes,Sun,Dinner,5
179,34.63,3.55,Male,Yes,Sun,Dinner,2
31,18.35,2.50,Male,No,Sat,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
71,17.07,3.00,Female,No,Sat,Dinner,3
6,8.77,2.00,Male,No,Sun,Dinner,2
95,40.17,4.73,Male,Yes,Fri,Dinner,4
131,20.27,2.83,Female,No,Thur,Lunch,2
157,25.00,3.75,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4


In [28]:
# total_bill 열의 값 4개를 임의로 선택하여 누락값으로 바꾸기
tips_10.loc[np.random.permutation(tips_10.index)[:4],'total_bill'] = np.NaN
tips_10

,total_bill,tip,sex,smoker,day,time,size
187,30.46,2.00,Male,Yes,Sun,Dinner,5
179,34.63,3.55,Male,Yes,Sun,Dinner,2
31,NaN,2.50,Male,No,Sat,Dinner,4
52,NaN,5.20,Female,No,Sun,Dinner,4
71,NaN,3.00,Female,No,Sat,Dinner,3
6,8.77,2.00,Male,No,Sun,Dinner,2
95,40.17,4.73,Male,Yes,Fri,Dinner,4
131,20.27,2.83,Female,No,Thur,Lunch,2
157,25.00,3.75,Female,No,Sun,Dinner,4
5,NaN,4.71,Male,No,Sun,Dinner,4


In [37]:
# 흡연자와 비흡연자 평균을 따로 계산
def fill_na_mean(x) :
    avg = x.mean()
    return x.fillna(avg)

tb_grp_mean = tips_10.groupby('smoker').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = round(tb_grp_mean, 2)
tips_10

,total_bill,tip,sex,smoker,day,time,size,fill_total_bill
187,30.46,2.00,Male,Yes,Sun,Dinner,5,30.46
179,34.63,3.55,Male,Yes,Sun,Dinner,2,34.63
31,NaN,2.50,Male,No,Sat,Dinner,4,18.01
52,NaN,5.20,Female,No,Sun,Dinner,4,18.01
71,NaN,3.00,Female,No,Sat,Dinner,3,18.01
6,8.77,2.00,Male,No,Sun,Dinner,2,8.77
95,40.17,4.73,Male,Yes,Fri,Dinner,4,40.17
131,20.27,2.83,Female,No,Thur,Lunch,2,20.27
157,25.00,3.75,Female,No,Sun,Dinner,4,25.00
5,NaN,4.71,Male,No,Sun,Dinner,4,18.01


#### 숫자로만 구성된 데이터프레임 ( apply, agg, tramsform )

In [38]:
def plus(x) :
    return x + 2

import pandas as pd
df = pd.DataFrame({'a':[10,20,30], 'b':[20,30,40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [39]:
print(df.apply(plus))
print(df.agg(plus))
print(df.transform(plus))

    a   b
0  12  22
1  22  32
2  32  42
    a   b
0  12  22
1  22  32
2  32  42
    a   b
0  12  22
1  22  32
2  32  42


In [40]:
print(df.a.apply(plus))
print(df.a.agg(plus))
print(df.a.transform(plus))

0    12
1    22
2    32
Name: a, dtype: int64
0    12
1    22
2    32
Name: a, dtype: int64
0    12
1    22
2    32
Name: a, dtype: int64


* 적용시킬 사용자 함수 내에서 집계 함수 사용

In [47]:
def mean(x) :
    return x.mean()

In [48]:
print(df.apply(mean))
print(df.agg(mean))
# print(df.transform(mean))  # error

a    20.0
b    30.0
dtype: float64
a    20.0
b    30.0
dtype: float64


In [52]:
df.a

0    10
1    20
2    30
Name: a, dtype: int64

In [49]:
# print(df.a.apply(mean))      # error
print(df.a.agg(mean)) 
# print(df.a.transform(mean))  # error
# 최소 2개 이상의 데이터가 있어야 집계 가능

20.0


* 숫자 데이터열과 문자데이터열이 함께 이쓴 데이터프레임

In [53]:
df2 = pd.DataFrame({
    'a' : [10,20,30,40],
    'b' : [20,30,40,50],
    'c' : ['서울','대전','부산','제주'],
    'd' : ['Female','Male','Male','Female'],
})

df2

,a,b,c,d
0,10,20,서울,Female
1,20,30,대전,Male
2,30,40,부산,Male
3,40,50,제주,Female


In [58]:
# 숫자 데이터와 문자데이터가 있는 경우 집계 불가능, 열 분리해서 진행해야함
# print(df2.apply(mean))
# print(df2.agg(mean))
# print(df2.transform(mean))
# print(df2.apply(plus))
# print(df2.agg(plus))
# print(df2.transform(plus))

* 그룹화한 데이터

In [62]:
df2_grp = df2.groupby('c')

# print(df2_grp.apply(plus))
# print(df2_grp.agg(mean))
# print(df2_grp.transform(mean))